# Importing Libraries

In [4]:
# Importing pre-trained model
from keras.applications import ResNet50V2

# Importing important layers of Keras
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.callbacks import EarlyStopping

# Importing data functions
from data_library.data_processing import black_white, crop, delete_face_images, encode, to_cat

# Importing preprocessing functions
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical


SymbolAlreadyExposedError: Symbol Zeros is already exposed as ().

# Importing the pre-trained model

In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

# Saving the model in a variable
resnet = ResNet50V2(weights='imagenet', include_top=False)


# 10 Classes from Real Madrid (cropped_faces)

In [ ]:
faces_folder = "../data_processing/raw_data/faces/"
cropped_faces_directory = "../data_processing/raw_data/cropped_faces"


In [ ]:
# # RUN ONLY IF YOU DONT HAVE THE CROPPED FACES YET
# black_white(faces_folder)
# crop(faces_folder, cropped_faces_directory)
# delete_face_images(cropped_faces_directory)


In [ ]:
y, X = encode(cropped_faces_directory)
y_cat = to_cat(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2)


## Model 1

In [ ]:
# Add a new classification head
layer = resnet.output
layer = GlobalAveragePooling2D()(layer)
predictions = Dense(y_train.shape[1], activation='softmax')(layer)

model = Model(inputs=resnet.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
es = EarlyStopping(patience=3,
                   restore_best_weights=True)

# Train the model
model.fit(X_train,
          y_train,
          epochs=10,
          batch_size=16,
          validation_split=0.2,
          callbacks=[es]
         )


Epoch 1/10


2023-11-25 13:07:30.926190: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


10/10 [==============================] - 5s 228ms/step - loss: 2.3318 - accuracy: 0.2756 - val_loss: 40.7103 - val_accuracy: 0.1250
Epoch 2/10
10/10 [==============================] - 2s 175ms/step - loss: 1.5321 - accuracy: 0.5256 - val_loss: 18.2984 - val_accuracy: 0.1250
Epoch 3/10
10/10 [==============================] - 2s 177ms/step - loss: 1.1933 - accuracy: 0.6474 - val_loss: 11.1928 - val_accuracy: 0.1250
Epoch 4/10
10/10 [==============================] - 2s 185ms/step - loss: 0.9986 - accuracy: 0.7500 - val_loss: 7.7443 - val_accuracy: 0.1250
Epoch 5/10
10/10 [==============================] - 2s 191ms/step - loss: 0.5227 - accuracy: 0.8590 - val_loss: 6.5362 - val_accuracy: 0.2250
Epoch 6/10
10/10 [==============================] - 2s 200ms/step - loss: 0.3563 - accuracy: 0.8974 - val_loss: 4.0324 - val_accuracy: 0.3000
Epoch 7/10
10/10 [==============================] - 2s 197ms/step - loss: 0.3021 - accuracy: 0.9167 - val_loss: 3.2774 - val_accuracy: 0.3250
Epoch 8/10
10/

In [ ]:
model.evaluate(X_test, y_test)


2/2 [==============================] - 0s 58ms/step - loss: 3.6748 - accuracy: 0.5714


[3.6748368740081787, 0.5714285969734192]

## Model 2

In [ ]:
# Define the input shape for your images
input_shape = X_train.shape[1:]

# Define a Sequential model or Functional API model
input_layer = Input(shape=input_shape)

# Add augmentation layers directly within the model
augmented = Conv2D(32, (3, 3), activation='relu')(input_layer)
augmented = MaxPooling2D((2, 2))(augmented)
# Add more layers or augmentation techniques as needed

# Rest of your model architecture
# Example:
augmented = Flatten()(augmented)
augmented = Dense(128, activation='relu')(augmented)
output_layer = Dense(y_train.shape[1], activation='softmax')(augmented)


model_2 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
es = EarlyStopping(patience=3,
                   restore_best_weights=True)

datagen = ImageDataGenerator(
    rotation_range=40,       # Rotate images by a wider range of degrees
    width_shift_range=0.3,   # Shift images horizontally by a larger fraction of total width
    height_shift_range=0.3,  # Shift images vertically by a larger fraction of total height
    shear_range=0.3,         # Apply a larger shear-based transformation
    zoom_range=0.3,          # Zoom in or out of images more aggressively
    horizontal_flip=True,    # Flip images horizontally
    vertical_flip=True,      # Flip images vertically
    fill_mode='nearest'      # Fill points outside the boundaries using the nearest available value
)

# Fit the ImageDataGenerator to your data
datagen.fit(X_train)

# Example of using the ImageDataGenerator during model training
model_2.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=100)


Epoch 1/100
7/7 [==============================] - 0s 16ms/step - loss: 721.4812 - accuracy: 0.0918
Epoch 2/100
7/7 [==============================] - 0s 15ms/step - loss: 444.1785 - accuracy: 0.0714
Epoch 3/100
7/7 [==============================] - 0s 15ms/step - loss: 153.8839 - accuracy: 0.1071
Epoch 4/100
7/7 [==============================] - 0s 15ms/step - loss: 81.5715 - accuracy: 0.0867
Epoch 5/100
7/7 [==============================] - 0s 14ms/step - loss: 29.0779 - accuracy: 0.1480
Epoch 6/100
7/7 [==============================] - 0s 16ms/step - loss: 16.8208 - accuracy: 0.1429
Epoch 7/100
7/7 [==============================] - 0s 15ms/step - loss: 9.2770 - accuracy: 0.1735
Epoch 8/100
7/7 [==============================] - 0s 15ms/step - loss: 5.8155 - accuracy: 0.1531
Epoch 9/100
7/7 [==============================] - 0s 15ms/step - loss: 4.1523 - accuracy: 0.1480
Epoch 10/100
7/7 [==============================] - 0s 15ms/step - loss: 3.7298 - accuracy: 0.1173
Epoch 11/1

In [ ]:
model_2.evaluate(X_test, y_test)


2/2 [==============================] - 0s 4ms/step - loss: 2.1795 - accuracy: 0.2857


[2.1795172691345215, 0.2857142984867096]

## Model 3

In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the input shape for your images
input_shape = X_train.shape[1:]

# Define a Sequential model or Functional API model
input_layer = Input(shape=input_shape)

# Add convolutional layers with pooling
conv1 = Conv2D(32, (3, 3), activation='relu')(input_layer)
pool1 = MaxPooling2D((2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu')(pool1)
pool2 = MaxPooling2D((2, 2))(conv2)

# Flatten the output before feeding into dense layers
flatten = Flatten()(pool2)

# Add dense layers with dropout for regularization
dense1 = Dense(128, activation='relu')(flatten)
dropout1 = Dropout(0.5)(dense1)  # Dropout layer for regularization

dense2 = Dense(64, activation='relu')(dropout1)
dropout2 = Dropout(0.5)(dense2)  # Dropout layer for regularization

# Output layer with softmax activation for multi-class classification
output_layer = Dense(y_train.shape[1], activation='softmax')(dropout2)

# Create the model
model_3 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Create an ImageDataGenerator with multiple augmentation techniques
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5],  # Vary brightness
    channel_shift_range=50,        # Vary channel shifts
    fill_mode='nearest'
)

# Fit the ImageDataGenerator to your data
datagen.fit(X_train)

# Example of using the ImageDataGenerator during model training
model_3.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=100)


Epoch 1/100
7/7 [==============================] - 0s 21ms/step - loss: 180.3172 - accuracy: 0.0714
Epoch 2/100
7/7 [==============================] - 0s 26ms/step - loss: 8.8773 - accuracy: 0.0969
Epoch 3/100
7/7 [==============================] - 0s 25ms/step - loss: 2.8235 - accuracy: 0.1531
Epoch 4/100
7/7 [==============================] - 0s 20ms/step - loss: 2.3463 - accuracy: 0.1020
Epoch 5/100
7/7 [==============================] - 0s 21ms/step - loss: 2.3244 - accuracy: 0.0867
Epoch 6/100
7/7 [==============================] - 0s 21ms/step - loss: 2.2962 - accuracy: 0.1327
Epoch 7/100
7/7 [==============================] - 0s 21ms/step - loss: 2.2856 - accuracy: 0.1173
Epoch 8/100
7/7 [==============================] - 0s 21ms/step - loss: 2.3175 - accuracy: 0.1429
Epoch 9/100
7/7 [==============================] - 0s 21ms/step - loss: 2.3045 - accuracy: 0.1122
Epoch 10/100
7/7 [==============================] - 0s 24ms/step - loss: 2.3059 - accuracy: 0.1327
Epoch 11/100
7/7 

In [ ]:
model_3.evaluate(X_test, y_test)


2/2 [==============================] - 0s 4ms/step - loss: 2.2999 - accuracy: 0.0612


[2.299858808517456, 0.06122449040412903]

## Model 4

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

model_4 = Sequential()
model_4.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model_4.add(MaxPooling2D((2, 2)))
model_4.add(Conv2D(64, (3, 3), activation='relu'))
model_4.add(MaxPooling2D((2, 2)))
model_4.add(Conv2D(128, (3, 3), activation='relu'))
model_4.add(MaxPooling2D((2, 2)))
model_4.add(Flatten())
model_4.add(Dense(256, activation='relu', kernel_regularizer='l2'))  # Adding L2 regularization
model_4.add(Dropout(0.5))  # Adding Dropout layer
model_4.add(Dense(128, activation='relu', kernel_regularizer='l2'))  # Adding L2 regularization
model_4.add(Dense(y_train.shape[1], activation='softmax'))

# Using Adam optimizer with a reduced learning rate
optimizer = Adam(lr=0.0001)
model_4.compile(optimizer="adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Adding more callbacks, e.g., ReduceLROnPlateau
callbacks = [EarlyStopping(patience=5, restore_best_weights=True),
             ReduceLROnPlateau(factor=0.1, patience=3)]

# Fit the model with increased complexity and callbacks
history = model_4.fit(datagen.flow(X_train, y_train, batch_size=16),
                    epochs=100,
                    callbacks=callbacks,
                    validation_data=(X_test, y_test))

model_4.evaluate(X_test, y_test)


Epoch 1/100


/Users/yassinebouaine/.pyenv/versions/3.10.6/envs/dynamic-players-insights/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


13/13 [==============================] - 1s 24ms/step - loss: 29.1012 - accuracy: 0.0867 - val_loss: 7.8460 - val_accuracy: 0.0612 - lr: 0.0010
Epoch 2/100
13/13 [==============================] - 0s 16ms/step - loss: 7.2830 - accuracy: 0.1276 - val_loss: 6.8762 - val_accuracy: 0.0612 - lr: 0.0010
Epoch 3/100
13/13 [==============================] - 0s 16ms/step - loss: 6.7763 - accuracy: 0.1122 - val_loss: 6.6517 - val_accuracy: 0.1429 - lr: 0.0010
Epoch 4/100
13/13 [==============================] - 0s 17ms/step - loss: 6.5523 - accuracy: 0.1276 - val_loss: 6.5493 - val_accuracy: 0.0816 - lr: 0.0010
Epoch 5/100
13/13 [==============================] - 0s 17ms/step - loss: 6.4302 - accuracy: 0.1224 - val_loss: 6.3743 - val_accuracy: 0.1224 - lr: 0.0010
Epoch 6/100
13/13 [==============================] - 0s 18ms/step - loss: 6.3092 - accuracy: 0.1173 - val_loss: 6.2687 - val_accuracy: 0.0612 - lr: 0.0010
Epoch 7/100
13/13 [==============================] - 0s 17ms/step - loss: 6.1862 

[3.493682861328125, 0.3877550959587097]

## Model 5

In [ ]:
from keras.layers import BatchNormalization, LeakyReLU

model_5 = Sequential()
model_5.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model_5.add(MaxPooling2D((2, 2)))
model_5.add(BatchNormalization())  # Add BatchNormalization after each Conv layer
model_5.add(Conv2D(64, (3, 3), activation='relu'))
model_5.add(MaxPooling2D((2, 2)))
model_5.add(BatchNormalization())
model_5.add(Conv2D(128, (3, 3), activation='relu'))
model_5.add(MaxPooling2D((2, 2)))
model_5.add(BatchNormalization())
model_5.add(Flatten())
model_5.add(Dense(256))
model_5.add(LeakyReLU(alpha=0.1))  # LeakyReLU activation
model_5.add(Dropout(0.5))
model_5.add(Dense(128))
model_5.add(LeakyReLU(alpha=0.1))
model_5.add(Dense(y_train.shape[1], activation='softmax'))

# Compile the model
model_5.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model with adjusted architecture
es = EarlyStopping(patience=10, restore_best_weights=True)
history = model_5.fit(datagen.flow(X_train, y_train, batch_size=64),
                    epochs=150,  # Train for more epochs
                    callbacks=es,
                    validation_data=(X_test, y_test))

model_5.evaluate(X_test, y_test)


Epoch 1/150
4/4 [==============================] - 1s 84ms/step - loss: 3.7428 - accuracy: 0.0765 - val_loss: 9.2981 - val_accuracy: 0.1224
Epoch 2/150
4/4 [==============================] - 0s 54ms/step - loss: 3.4164 - accuracy: 0.0918 - val_loss: 7.7855 - val_accuracy: 0.1020
Epoch 3/150
4/4 [==============================] - 0s 71ms/step - loss: 3.2929 - accuracy: 0.1224 - val_loss: 8.5709 - val_accuracy: 0.1429
Epoch 4/150
4/4 [==============================] - 0s 55ms/step - loss: 2.9323 - accuracy: 0.1582 - val_loss: 8.2315 - val_accuracy: 0.1020
Epoch 5/150
4/4 [==============================] - 0s 56ms/step - loss: 3.4078 - accuracy: 0.1378 - val_loss: 9.7438 - val_accuracy: 0.0408
Epoch 6/150
4/4 [==============================] - 0s 54ms/step - loss: 3.0643 - accuracy: 0.1837 - val_loss: 9.9507 - val_accuracy: 0.0204
Epoch 7/150
4/4 [==============================] - 0s 57ms/step - loss: 3.1997 - accuracy: 0.1888 - val_loss: 6.0693 - val_accuracy: 0.1020
Epoch 8/150
4/4 [===

[1.851344347000122, 0.36734694242477417]

# Video - 10 Classes (video_cropped_faces_0)

In [ ]:
video_cropped_faces_directory = "../data_processing/raw_data/video_cropped_faces_0"


In [ ]:
y, X = encode(video_cropped_faces_directory)
y_cat = to_cat(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2)


In [ ]:
y_train.shape


(63, 11)

## Model 1

In [ ]:
# Add a new classification head
layer = resnet.output
layer = GlobalAveragePooling2D()(layer)
predictions = Dense(y_train.shape[1], activation='softmax')(layer)

model_6 = Model(inputs=resnet.input, outputs=predictions)

# Compile the model
model_6.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
es = EarlyStopping(patience=3,
                   restore_best_weights=True)

# Train the model
model_6.fit(X_train,
          y_train,
          epochs=10,
          batch_size=16,
          validation_split=0.2,
          callbacks=[es]
         )


Epoch 1/10
4/4 [==============================] - 3s 297ms/step - loss: 1.9391 - accuracy: 0.4000 - val_loss: 2.3818 - val_accuracy: 0.6923
Epoch 2/10
4/4 [==============================] - 1s 155ms/step - loss: 0.9425 - accuracy: 0.7200 - val_loss: 16.2346 - val_accuracy: 0.4615
Epoch 3/10
4/4 [==============================] - 1s 156ms/step - loss: 0.9125 - accuracy: 0.8600 - val_loss: 12.6295 - val_accuracy: 0.4615
Epoch 4/10
4/4 [==============================] - 1s 163ms/step - loss: 0.4429 - accuracy: 0.9600 - val_loss: 33.6319 - val_accuracy: 0.1538


In [ ]:
model_6.evaluate(X_test, y_test)


1/1 [==============================] - 0s 61ms/step - loss: 3.6977 - accuracy: 0.5000


[3.6977198123931885, 0.5]

## Model 2

In [ ]:
# Define the input shape for your images
input_shape = X_train.shape[1:]

# Define a Sequential model or Functional API model
input_layer = Input(shape=input_shape)

# Add augmentation layers directly within the model
augmented = Conv2D(32, (3, 3), activation='relu')(input_layer)
augmented = MaxPooling2D((2, 2))(augmented)
# Add more layers or augmentation techniques as needed

# Rest of your model architecture
# Example:
augmented = Flatten()(augmented)
augmented = Dense(128, activation='relu')(augmented)
output_layer = Dense(y_train.shape[1], activation='softmax')(augmented)


model_7 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_7.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
es = EarlyStopping(patience=3,
                   restore_best_weights=True)

datagen = ImageDataGenerator(
    rotation_range=40,       # Rotate images by a wider range of degrees
    width_shift_range=0.3,   # Shift images horizontally by a larger fraction of total width
    height_shift_range=0.3,  # Shift images vertically by a larger fraction of total height
    shear_range=0.3,         # Apply a larger shear-based transformation
    zoom_range=0.3,          # Zoom in or out of images more aggressively
    horizontal_flip=True,    # Flip images horizontally
    vertical_flip=True,      # Flip images vertically
    fill_mode='nearest'      # Fill points outside the boundaries using the nearest available value
)

# Fit the ImageDataGenerator to your data
datagen.fit(X_train)

# Example of using the ImageDataGenerator during model training
model_7.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=100)


Epoch 1/100
2/2 [==============================] - 0s 19ms/step - loss: 353.8517 - accuracy: 0.0952
Epoch 2/100
2/2 [==============================] - 0s 15ms/step - loss: 988.5944 - accuracy: 0.2063
Epoch 3/100
2/2 [==============================] - 0s 15ms/step - loss: 659.9906 - accuracy: 0.1111
Epoch 4/100
2/2 [==============================] - 0s 14ms/step - loss: 465.5725 - accuracy: 0.1111
Epoch 5/100
2/2 [==============================] - 0s 15ms/step - loss: 463.4159 - accuracy: 0.0794
Epoch 6/100
2/2 [==============================] - 0s 16ms/step - loss: 388.7559 - accuracy: 0.2222
Epoch 7/100
2/2 [==============================] - 0s 15ms/step - loss: 293.9539 - accuracy: 0.1429
Epoch 8/100
2/2 [==============================] - 0s 16ms/step - loss: 213.7339 - accuracy: 0.0635
Epoch 9/100
2/2 [==============================] - 0s 15ms/step - loss: 146.0322 - accuracy: 0.1746
Epoch 10/100
2/2 [==============================] - 0s 14ms/step - loss: 113.4483 - accuracy: 0.1905

In [ ]:
model_7.evaluate(X_test, y_test)


1/1 [==============================] - 0s 69ms/step - loss: 3.0438 - accuracy: 0.5000


[3.0437562465667725, 0.5]

## Model 3

In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the input shape for your images
input_shape = X_train.shape[1:]

# Define a Sequential model or Functional API model
input_layer = Input(shape=input_shape)

# Add convolutional layers with pooling
conv1 = Conv2D(32, (3, 3), activation='relu')(input_layer)
pool1 = MaxPooling2D((2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu')(pool1)
pool2 = MaxPooling2D((2, 2))(conv2)

# Flatten the output before feeding into dense layers
flatten = Flatten()(pool2)

# Add dense layers with dropout for regularization
dense1 = Dense(128, activation='relu')(flatten)
dropout1 = Dropout(0.5)(dense1)  # Dropout layer for regularization

dense2 = Dense(64, activation='relu')(dropout1)
dropout2 = Dropout(0.5)(dense2)  # Dropout layer for regularization

# Output layer with softmax activation for multi-class classification
output_layer = Dense(y_train.shape[1], activation='softmax')(dropout2)

# Create the model
model_8 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_8.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Create an ImageDataGenerator with multiple augmentation techniques
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5],  # Vary brightness
    channel_shift_range=50,        # Vary channel shifts
    fill_mode='nearest'
)

# Fit the ImageDataGenerator to your data
datagen.fit(X_train)

# Example of using the ImageDataGenerator during model training
model_8.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=100)


Epoch 1/100
2/2 [==============================] - 0s 24ms/step - loss: 127.9656 - accuracy: 0.1111
Epoch 2/100
2/2 [==============================] - 0s 19ms/step - loss: 84.3686 - accuracy: 0.0476
Epoch 3/100
2/2 [==============================] - 0s 24ms/step - loss: 29.2708 - accuracy: 0.0476
Epoch 4/100
2/2 [==============================] - 0s 26ms/step - loss: 13.7281 - accuracy: 0.0476
Epoch 5/100
2/2 [==============================] - 0s 22ms/step - loss: 3.9468 - accuracy: 0.0317
Epoch 6/100
2/2 [==============================] - 0s 24ms/step - loss: 2.6466 - accuracy: 0.0476
Epoch 7/100
2/2 [==============================] - 0s 20ms/step - loss: 2.8317 - accuracy: 0.0952
Epoch 8/100
2/2 [==============================] - 0s 21ms/step - loss: 2.9847 - accuracy: 0.0635
Epoch 9/100
2/2 [==============================] - 0s 22ms/step - loss: 2.4616 - accuracy: 0.0635
Epoch 10/100
2/2 [==============================] - 0s 20ms/step - loss: 2.5157 - accuracy: 0.0794
Epoch 11/100
2

In [ ]:
model_8.evaluate(X_test, y_test)


1/1 [==============================] - 0s 78ms/step - loss: 2.3544 - accuracy: 0.0000e+00


[2.35441255569458, 0.0]

## Model 4

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

model_9 = Sequential()
model_9.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model_9.add(MaxPooling2D((2, 2)))
model_9.add(Conv2D(64, (3, 3), activation='relu'))
model_9.add(MaxPooling2D((2, 2)))
model_9.add(Conv2D(128, (3, 3), activation='relu'))
model_9.add(MaxPooling2D((2, 2)))
model_9.add(Flatten())
model_9.add(Dense(256, activation='relu', kernel_regularizer='l2'))  # Adding L2 regularization
model_9.add(Dropout(0.5))  # Adding Dropout layer
model_9.add(Dense(128, activation='relu', kernel_regularizer='l2'))  # Adding L2 regularization
model_9.add(Dense(y_train.shape[1], activation='softmax'))

# Using Adam optimizer with a reduced learning rate
optimizer = Adam(lr=0.0001)
model_9.compile(optimizer="adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Adding more callbacks, e.g., ReduceLROnPlateau
callbacks = [EarlyStopping(patience=5, restore_best_weights=True),
             ReduceLROnPlateau(factor=0.1, patience=3)]

# Fit the model with increased complexity and callbacks
history = model_9.fit(datagen.flow(X_train, y_train, batch_size=16),
                    epochs=100,
                    callbacks=callbacks,
                    validation_data=(X_test, y_test))


Epoch 1/100


/Users/yassinebouaine/.pyenv/versions/3.10.6/envs/dynamic-players-insights/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


4/4 [==============================] - 1s 47ms/step - loss: 19.9186 - accuracy: 0.1429 - val_loss: 11.2547 - val_accuracy: 0.0625 - lr: 0.0010
Epoch 2/100
4/4 [==============================] - 0s 20ms/step - loss: 10.1309 - accuracy: 0.1587 - val_loss: 7.7378 - val_accuracy: 0.2500 - lr: 0.0010
Epoch 3/100
4/4 [==============================] - 0s 19ms/step - loss: 7.9676 - accuracy: 0.1587 - val_loss: 7.5848 - val_accuracy: 0.0625 - lr: 0.0010
Epoch 4/100
4/4 [==============================] - 0s 20ms/step - loss: 7.4027 - accuracy: 0.2063 - val_loss: 7.1942 - val_accuracy: 0.2500 - lr: 0.0010
Epoch 5/100
4/4 [==============================] - 0s 19ms/step - loss: 7.1223 - accuracy: 0.1587 - val_loss: 7.0266 - val_accuracy: 0.1875 - lr: 0.0010
Epoch 6/100
4/4 [==============================] - 0s 19ms/step - loss: 6.8365 - accuracy: 0.2381 - val_loss: 6.8746 - val_accuracy: 0.3125 - lr: 0.0010
Epoch 7/100
4/4 [==============================] - 0s 19ms/step - loss: 6.6343 - accuracy: 

In [ ]:
model_9.evaluate(X_test, y_test)


1/1 [==============================] - 0s 14ms/step - loss: 5.1622 - accuracy: 0.3125


[5.162161827087402, 0.3125]

## Model 5

In [ ]:
from keras.layers import BatchNormalization, LeakyReLU

model_10 = Sequential()
model_10.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(BatchNormalization())  # Add BatchNormalization after each Conv layer
model_10.add(Conv2D(64, (3, 3), activation='relu'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(BatchNormalization())
model_10.add(Conv2D(128, (3, 3), activation='relu'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(BatchNormalization())
model_10.add(Flatten())
model_10.add(Dense(256))
model_10.add(LeakyReLU(alpha=0.1))  # LeakyReLU activation
model_10.add(Dropout(0.5))
model_10.add(Dense(128))
model_10.add(LeakyReLU(alpha=0.1))
model_10.add(Dense(y_train.shape[1], activation='softmax'))

# Compile the model
model_10.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model with adjusted architecture
es = EarlyStopping(patience=10, restore_best_weights=True)
history = model_10.fit(datagen.flow(X_train, y_train, batch_size=64),
                    epochs=150,  # Train for more epochs
                    callbacks=es,
                    validation_data=(X_test, y_test))


Epoch 1/150
1/1 [==============================] - 1s 520ms/step - loss: 2.9791 - accuracy: 0.0159 - val_loss: 17.7261 - val_accuracy: 0.0625
Epoch 2/150
1/1 [==============================] - 0s 124ms/step - loss: 2.7497 - accuracy: 0.1746 - val_loss: 8.0909 - val_accuracy: 0.2500
Epoch 3/150
1/1 [==============================] - 0s 113ms/step - loss: 3.1191 - accuracy: 0.1429 - val_loss: 4.6514 - val_accuracy: 0.2500
Epoch 4/150
1/1 [==============================] - 0s 108ms/step - loss: 2.7099 - accuracy: 0.2540 - val_loss: 4.9547 - val_accuracy: 0.1875
Epoch 5/150
1/1 [==============================] - 0s 106ms/step - loss: 2.4965 - accuracy: 0.2857 - val_loss: 4.6919 - val_accuracy: 0.1875
Epoch 6/150
1/1 [==============================] - 0s 108ms/step - loss: 2.4703 - accuracy: 0.1905 - val_loss: 5.5271 - val_accuracy: 0.0000e+00
Epoch 7/150
1/1 [==============================] - 0s 105ms/step - loss: 2.7935 - accuracy: 0.3175 - val_loss: 5.8005 - val_accuracy: 0.2500
Epoch 8/

In [ ]:
model_10.evaluate(X_test, y_test)


1/1 [==============================] - 0s 20ms/step - loss: 3.1046 - accuracy: 0.3750


[3.1046385765075684, 0.375]

# Video + Getty - 10 Classes (video_cropped_faces_1)

In [ ]:
cropped_faces_directory = "../data_processing/raw_data/video_cropped_faces_1"


In [ ]:
y, X = encode(cropped_faces_directory)
y_cat = to_cat(y)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2)


## Model 1

In [ ]:
# Add a new classification head
layer = resnet.output
layer = GlobalAveragePooling2D()(layer)
predictions = Dense(y_train.shape[1], activation='softmax')(layer)

model_11 = Model(inputs=resnet.input, outputs=predictions)

# Compile the model
model_11.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
es = EarlyStopping(patience=3,
                   restore_best_weights=True)

# Train the model
model_11.fit(X_train,
          y_train,
          epochs=10,
          batch_size=16,
          validation_split=0.2,
          callbacks=[es]
         )


Epoch 1/10
13/13 [==============================] - 5s 215ms/step - loss: 2.3762 - accuracy: 0.2598 - val_loss: 450.3246 - val_accuracy: 0.1923
Epoch 2/10
13/13 [==============================] - 2s 176ms/step - loss: 2.0315 - accuracy: 0.4216 - val_loss: 135.6507 - val_accuracy: 0.1731
Epoch 3/10
13/13 [==============================] - 2s 177ms/step - loss: 1.1117 - accuracy: 0.6618 - val_loss: 188.9164 - val_accuracy: 0.0962
Epoch 4/10
13/13 [==============================] - 2s 190ms/step - loss: 0.8147 - accuracy: 0.7353 - val_loss: 95.1989 - val_accuracy: 0.1154
Epoch 5/10
13/13 [==============================] - 2s 182ms/step - loss: 0.6117 - accuracy: 0.7745 - val_loss: 92.2703 - val_accuracy: 0.2115
Epoch 6/10
13/13 [==============================] - 3s 194ms/step - loss: 0.8906 - accuracy: 0.7157 - val_loss: 53.0513 - val_accuracy: 0.1538
Epoch 7/10
13/13 [==============================] - 3s 208ms/step - loss: 0.5180 - accuracy: 0.8431 - val_loss: 30.7575 - val_accuracy: 0.1

In [ ]:
model_11.evaluate(X_test, y_test)


2/2 [==============================] - 0s 70ms/step - loss: 18.0831 - accuracy: 0.2188


[18.08306312561035, 0.21875]

## Model 2

In [ ]:
# Define the input shape for your images
input_shape = X_train.shape[1:]

# Define a Sequential model or Functional API model
input_layer = Input(shape=input_shape)

# Add augmentation layers directly within the model
augmented = Conv2D(32, (3, 3), activation='relu')(input_layer)
augmented = MaxPooling2D((2, 2))(augmented)
# Add more layers or augmentation techniques as needed

# Rest of your model architecture
# Example:
augmented = Flatten()(augmented)
augmented = Dense(128, activation='relu')(augmented)
output_layer = Dense(y_train.shape[1], activation='softmax')(augmented)


model_12 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_12.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
es = EarlyStopping(patience=3,
                   restore_best_weights=True)

datagen = ImageDataGenerator(
    rotation_range=40,       # Rotate images by a wider range of degrees
    width_shift_range=0.3,   # Shift images horizontally by a larger fraction of total width
    height_shift_range=0.3,  # Shift images vertically by a larger fraction of total height
    shear_range=0.3,         # Apply a larger shear-based transformation
    zoom_range=0.3,          # Zoom in or out of images more aggressively
    horizontal_flip=True,    # Flip images horizontally
    vertical_flip=True,      # Flip images vertically
    fill_mode='nearest'      # Fill points outside the boundaries using the nearest available value
)

# Fit the ImageDataGenerator to your data
datagen.fit(X_train)

# Example of using the ImageDataGenerator during model training
model_12.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=100)


Epoch 1/100
8/8 [==============================] - 0s 16ms/step - loss: 770.1324 - accuracy: 0.1133
Epoch 2/100
8/8 [==============================] - 0s 15ms/step - loss: 312.3860 - accuracy: 0.1094
Epoch 3/100
8/8 [==============================] - 0s 15ms/step - loss: 98.2151 - accuracy: 0.1094
Epoch 4/100
8/8 [==============================] - 0s 15ms/step - loss: 31.4954 - accuracy: 0.1250
Epoch 5/100
8/8 [==============================] - 0s 16ms/step - loss: 8.4596 - accuracy: 0.1289
Epoch 6/100
8/8 [==============================] - 0s 17ms/step - loss: 3.7146 - accuracy: 0.1484
Epoch 7/100
8/8 [==============================] - 0s 16ms/step - loss: 2.8657 - accuracy: 0.1562
Epoch 8/100
8/8 [==============================] - 0s 16ms/step - loss: 2.6987 - accuracy: 0.1602
Epoch 9/100
8/8 [==============================] - 0s 15ms/step - loss: 2.4344 - accuracy: 0.1836
Epoch 10/100
8/8 [==============================] - 0s 15ms/step - loss: 2.4472 - accuracy: 0.1758
Epoch 11/100


In [ ]:
model_12.evaluate(X_test, y_test)


2/2 [==============================] - 0s 4ms/step - loss: 2.8802 - accuracy: 0.2656


[2.880157709121704, 0.265625]

## Model 3

In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the input shape for your images
input_shape = X_train.shape[1:]

# Define a Sequential model or Functional API model
input_layer = Input(shape=input_shape)

# Add convolutional layers with pooling
conv1 = Conv2D(32, (3, 3), activation='relu')(input_layer)
pool1 = MaxPooling2D((2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu')(pool1)
pool2 = MaxPooling2D((2, 2))(conv2)

# Flatten the output before feeding into dense layers
flatten = Flatten()(pool2)

# Add dense layers with dropout for regularization
dense1 = Dense(128, activation='relu')(flatten)
dropout1 = Dropout(0.5)(dense1)  # Dropout layer for regularization

dense2 = Dense(64, activation='relu')(dropout1)
dropout2 = Dropout(0.5)(dense2)  # Dropout layer for regularization

# Output layer with softmax activation for multi-class classification
output_layer = Dense(y_train.shape[1], activation='softmax')(dropout2)

# Create the model
model_13 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_13.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Create an ImageDataGenerator with multiple augmentation techniques
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5],  # Vary brightness
    channel_shift_range=50,        # Vary channel shifts
    fill_mode='nearest'
)

# Fit the ImageDataGenerator to your data
datagen.fit(X_train)

# Example of using the ImageDataGenerator during model training
model_13.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=100)


Epoch 1/100
8/8 [==============================] - 0s 23ms/step - loss: 68.1864 - accuracy: 0.0977
Epoch 2/100
8/8 [==============================] - 0s 31ms/step - loss: 4.2623 - accuracy: 0.0781
Epoch 3/100
8/8 [==============================] - 0s 22ms/step - loss: 2.4763 - accuracy: 0.0625
Epoch 4/100
8/8 [==============================] - 0s 24ms/step - loss: 2.3128 - accuracy: 0.0977
Epoch 5/100
8/8 [==============================] - 0s 28ms/step - loss: 2.3074 - accuracy: 0.0938
Epoch 6/100
8/8 [==============================] - 0s 23ms/step - loss: 2.2991 - accuracy: 0.1016
Epoch 7/100
8/8 [==============================] - 0s 24ms/step - loss: 2.2798 - accuracy: 0.0820
Epoch 8/100
8/8 [==============================] - 0s 22ms/step - loss: 2.2926 - accuracy: 0.1328
Epoch 9/100
8/8 [==============================] - 0s 22ms/step - loss: 2.2574 - accuracy: 0.1602
Epoch 10/100
8/8 [==============================] - 0s 26ms/step - loss: 2.2608 - accuracy: 0.1367
Epoch 11/100
8/8 [

In [ ]:
model_13.evaluate(X_test, y_test)


2/2 [==============================] - 0s 6ms/step - loss: 4.1512 - accuracy: 0.1719


[4.151215553283691, 0.171875]

## Model 4

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

model_14 = Sequential()
model_14.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model_14.add(MaxPooling2D((2, 2)))
model_14.add(Conv2D(64, (3, 3), activation='relu'))
model_14.add(MaxPooling2D((2, 2)))
model_14.add(Conv2D(128, (3, 3), activation='relu'))
model_14.add(MaxPooling2D((2, 2)))
model_14.add(Flatten())
model_14.add(Dense(256, activation='relu', kernel_regularizer='l2'))  # Adding L2 regularization
model_14.add(Dropout(0.5))  # Adding Dropout layer
model_14.add(Dense(128, activation='relu', kernel_regularizer='l2'))  # Adding L2 regularization
model_14.add(Dense(y_train.shape[1], activation='softmax'))

# Using Adam optimizer with a reduced learning rate
optimizer = Adam(lr=0.0001)
model_14.compile(optimizer="adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Adding more callbacks, e.g., ReduceLROnPlateau
callbacks = [EarlyStopping(patience=5, restore_best_weights=True),
             ReduceLROnPlateau(factor=0.1, patience=3)]

# Fit the model with increased complexity and callbacks
history = model_14.fit(datagen.flow(X_train, y_train, batch_size=16),
                    epochs=100,
                    callbacks=callbacks,
                    validation_data=(X_test, y_test))


Epoch 1/100


/Users/yassinebouaine/.pyenv/versions/3.10.6/envs/dynamic-players-insights/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


16/16 [==============================] - 1s 22ms/step - loss: 17.1474 - accuracy: 0.0977 - val_loss: 7.2742 - val_accuracy: 0.1562 - lr: 0.0010
Epoch 2/100
16/16 [==============================] - 0s 15ms/step - loss: 7.0583 - accuracy: 0.1445 - val_loss: 6.8243 - val_accuracy: 0.1250 - lr: 0.0010
Epoch 3/100
16/16 [==============================] - 0s 16ms/step - loss: 6.6609 - accuracy: 0.1094 - val_loss: 6.5463 - val_accuracy: 0.0781 - lr: 0.0010
Epoch 4/100
16/16 [==============================] - 0s 16ms/step - loss: 6.4437 - accuracy: 0.1055 - val_loss: 6.3237 - val_accuracy: 0.0781 - lr: 0.0010
Epoch 5/100
16/16 [==============================] - 0s 16ms/step - loss: 6.2176 - accuracy: 0.1016 - val_loss: 6.1335 - val_accuracy: 0.0781 - lr: 0.0010
Epoch 6/100
16/16 [==============================] - 0s 16ms/step - loss: 6.0335 - accuracy: 0.1328 - val_loss: 5.9608 - val_accuracy: 0.0625 - lr: 0.0010
Epoch 7/100
16/16 [==============================] - 0s 21ms/step - loss: 5.8592 

In [ ]:
model_14.evaluate(X_test, y_test)


2/2 [==============================] - 0s 7ms/step - loss: 4.4757 - accuracy: 0.1406


[4.4756550788879395, 0.140625]

## Model 5

In [ ]:
from keras.layers import BatchNormalization, LeakyReLU

model_15 = Sequential()
model_15.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model_15.add(MaxPooling2D((2, 2)))
model_15.add(BatchNormalization())  # Add BatchNormalization after each Conv layer
model_15.add(Conv2D(64, (3, 3), activation='relu'))
model_15.add(MaxPooling2D((2, 2)))
model_15.add(BatchNormalization())
model_15.add(Conv2D(128, (3, 3), activation='relu'))
model_15.add(MaxPooling2D((2, 2)))
model_15.add(BatchNormalization())
model_15.add(Flatten())
model_15.add(Dense(256))
model_15.add(LeakyReLU(alpha=0.1))  # LeakyReLU activation
model_15.add(Dropout(0.5))
model_15.add(Dense(128))
model_15.add(LeakyReLU(alpha=0.1))
model_15.add(Dense(y_train.shape[1], activation='softmax'))

# Compile the model
model_15.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model with adjusted architecture
es = EarlyStopping(patience=10, restore_best_weights=True)
history = model_15.fit(datagen.flow(X_train, y_train, batch_size=64),
                    epochs=150,  # Train for more epochs
                    callbacks=es,
                    validation_data=(X_test, y_test))


Epoch 1/150
4/4 [==============================] - 1s 181ms/step - loss: 3.1718 - accuracy: 0.1055 - val_loss: 4.8802 - val_accuracy: 0.1406
Epoch 2/150
4/4 [==============================] - 0s 59ms/step - loss: 3.4431 - accuracy: 0.1133 - val_loss: 7.9642 - val_accuracy: 0.0938
Epoch 3/150
4/4 [==============================] - 0s 57ms/step - loss: 3.4310 - accuracy: 0.1680 - val_loss: 5.9525 - val_accuracy: 0.0781
Epoch 4/150
4/4 [==============================] - 0s 62ms/step - loss: 3.0578 - accuracy: 0.1523 - val_loss: 4.0330 - val_accuracy: 0.0781
Epoch 5/150
4/4 [==============================] - 0s 57ms/step - loss: 3.1684 - accuracy: 0.1328 - val_loss: 3.0378 - val_accuracy: 0.1250
Epoch 6/150
4/4 [==============================] - 0s 57ms/step - loss: 2.9309 - accuracy: 0.1602 - val_loss: 3.3828 - val_accuracy: 0.1250
Epoch 7/150
4/4 [==============================] - 0s 57ms/step - loss: 2.9117 - accuracy: 0.1641 - val_loss: 2.9726 - val_accuracy: 0.1094
Epoch 8/150
4/4 [==

In [ ]:
model_15.evaluate(X_test, y_test)


2/2 [==============================] - 0s 10ms/step - loss: 2.2823 - accuracy: 0.1719


[2.2823073863983154, 0.171875]

# Video + Getty - 5 Classes (video_cropped_faces_2)

In [ ]:
cropped_faces_directory = "../data_processing/raw_data/video_cropped_faces_2"

y, X = encode(cropped_faces_directory)
y_cat = to_cat(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2)


## Model 1

In [ ]:
# Add a new classification head
layer = resnet.output
layer = GlobalAveragePooling2D()(layer)
predictions = Dense(y_train.shape[1], activation='softmax')(layer)

model_16 = Model(inputs=resnet.input, outputs=predictions)

# Compile the model
model_16.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
es = EarlyStopping(patience=3,
                   restore_best_weights=True)

# Train the model
model_16.fit(X_train,
          y_train,
          epochs=10,
          batch_size=16,
          validation_split=0.2,
          callbacks=[es]
         )

model_16.evaluate(X_test, y_test)


Epoch 1/10
8/8 [==============================] - 4s 234ms/step - loss: 0.8407 - accuracy: 0.6810 - val_loss: 15.1895 - val_accuracy: 0.3793
Epoch 2/10
8/8 [==============================] - 1s 166ms/step - loss: 0.5384 - accuracy: 0.8966 - val_loss: 68.8213 - val_accuracy: 0.3793
Epoch 3/10
8/8 [==============================] - 1s 178ms/step - loss: 0.2376 - accuracy: 0.9310 - val_loss: 27.3002 - val_accuracy: 0.4483
Epoch 4/10
2/2 [==============================] - 0s 23ms/step - loss: 9.3170 - accuracy: 0.4865


[9.317042350769043, 0.4864864945411682]

## Model 2

In [ ]:
# Define the input shape for your images
input_shape = X_train.shape[1:]

# Define a Sequential model or Functional API model
input_layer = Input(shape=input_shape)

# Add augmentation layers directly within the model
augmented = Conv2D(32, (3, 3), activation='relu')(input_layer)
augmented = MaxPooling2D((2, 2))(augmented)
# Add more layers or augmentation techniques as needed

# Rest of your model architecture
# Example:
augmented = Flatten()(augmented)
augmented = Dense(128, activation='relu')(augmented)
output_layer = Dense(y_train.shape[1], activation='softmax')(augmented)


model_17 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_17.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
es = EarlyStopping(patience=3,
                   restore_best_weights=True)

datagen = ImageDataGenerator(
    rotation_range=40,       # Rotate images by a wider range of degrees
    width_shift_range=0.3,   # Shift images horizontally by a larger fraction of total width
    height_shift_range=0.3,  # Shift images vertically by a larger fraction of total height
    shear_range=0.3,         # Apply a larger shear-based transformation
    zoom_range=0.3,          # Zoom in or out of images more aggressively
    horizontal_flip=True,    # Flip images horizontally
    vertical_flip=True,      # Flip images vertically
    fill_mode='nearest'      # Fill points outside the boundaries using the nearest available value
)

# Fit the ImageDataGenerator to your data
datagen.fit(X_train)

# Example of using the ImageDataGenerator during model training
model_17.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=100)

model_17.evaluate(X_test, y_test)


Epoch 1/100
5/5 [==============================] - 1s 15ms/step - loss: 697.0805 - accuracy: 0.1862
Epoch 2/100
5/5 [==============================] - 0s 14ms/step - loss: 482.5548 - accuracy: 0.2069
Epoch 3/100
5/5 [==============================] - 0s 15ms/step - loss: 136.1004 - accuracy: 0.2000
Epoch 4/100
5/5 [==============================] - 0s 16ms/step - loss: 33.9276 - accuracy: 0.2069
Epoch 5/100
5/5 [==============================] - 0s 15ms/step - loss: 20.4791 - accuracy: 0.2069
Epoch 6/100
5/5 [==============================] - 0s 15ms/step - loss: 11.2673 - accuracy: 0.2207
Epoch 7/100
5/5 [==============================] - 0s 17ms/step - loss: 5.3486 - accuracy: 0.1931
Epoch 8/100
5/5 [==============================] - 0s 15ms/step - loss: 4.4486 - accuracy: 0.3103
Epoch 9/100
5/5 [==============================] - 0s 15ms/step - loss: 3.4048 - accuracy: 0.2897
Epoch 10/100
5/5 [==============================] - 0s 17ms/step - loss: 3.2501 - accuracy: 0.3517
Epoch 11/1

[1.6263487339019775, 0.45945945382118225]

## Model 3

In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the input shape for your images
input_shape = X_train.shape[1:]

# Define a Sequential model or Functional API model
input_layer = Input(shape=input_shape)

# Add convolutional layers with pooling
conv1 = Conv2D(32, (3, 3), activation='relu')(input_layer)
pool1 = MaxPooling2D((2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu')(pool1)
pool2 = MaxPooling2D((2, 2))(conv2)

# Flatten the output before feeding into dense layers
flatten = Flatten()(pool2)

# Add dense layers with dropout for regularization
dense1 = Dense(128, activation='relu')(flatten)
dropout1 = Dropout(0.5)(dense1)  # Dropout layer for regularization

dense2 = Dense(64, activation='relu')(dropout1)
dropout2 = Dropout(0.5)(dense2)  # Dropout layer for regularization

# Output layer with softmax activation for multi-class classification
output_layer = Dense(y_train.shape[1], activation='softmax')(dropout2)

# Create the model
model_18 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_18.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Create an ImageDataGenerator with multiple augmentation techniques
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5],  # Vary brightness
    channel_shift_range=50,        # Vary channel shifts
    fill_mode='nearest'
)

# Fit the ImageDataGenerator to your data
datagen.fit(X_train)

# Example of using the ImageDataGenerator during model training
model_18.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=100)

model_18.evaluate(X_test, y_test)


Epoch 1/100
5/5 [==============================] - 0s 23ms/step - loss: 135.2441 - accuracy: 0.2207
Epoch 2/100
5/5 [==============================] - 0s 20ms/step - loss: 38.2141 - accuracy: 0.2414
Epoch 3/100
5/5 [==============================] - 0s 23ms/step - loss: 6.4448 - accuracy: 0.1586
Epoch 4/100
5/5 [==============================] - 0s 21ms/step - loss: 2.6992 - accuracy: 0.1931
Epoch 5/100
5/5 [==============================] - 0s 19ms/step - loss: 1.7472 - accuracy: 0.2207
Epoch 6/100
5/5 [==============================] - 0s 21ms/step - loss: 1.6349 - accuracy: 0.2276
Epoch 7/100
5/5 [==============================] - 0s 24ms/step - loss: 1.6600 - accuracy: 0.2207
Epoch 8/100
5/5 [==============================] - 0s 20ms/step - loss: 1.7309 - accuracy: 0.2207
Epoch 9/100
5/5 [==============================] - 0s 22ms/step - loss: 1.6558 - accuracy: 0.1931
Epoch 10/100
5/5 [==============================] - 0s 18ms/step - loss: 1.6333 - accuracy: 0.1724
Epoch 11/100
5/5

[1.5990294218063354, 0.2432432472705841]

## Model 4

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

model_19 = Sequential()
model_19.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model_19.add(MaxPooling2D((2, 2)))
model_19.add(Conv2D(64, (3, 3), activation='relu'))
model_19.add(MaxPooling2D((2, 2)))
model_19.add(Conv2D(128, (3, 3), activation='relu'))
model_19.add(MaxPooling2D((2, 2)))
model_19.add(Flatten())
model_19.add(Dense(256, activation='relu', kernel_regularizer='l2'))  # Adding L2 regularization
model_19.add(Dropout(0.5))  # Adding Dropout layer
model_19.add(Dense(128, activation='relu', kernel_regularizer='l2'))  # Adding L2 regularization
model_19.add(Dense(y_train.shape[1], activation='softmax'))

# Using Adam optimizer with a reduced learning rate
optimizer = Adam(lr=0.0001)
model_19.compile(optimizer="adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Adding more callbacks, e.g., ReduceLROnPlateau
callbacks = [EarlyStopping(patience=5, restore_best_weights=True),
             ReduceLROnPlateau(factor=0.1, patience=3)]

# Fit the model with increased complexity and callbacks
history = model_19.fit(datagen.flow(X_train, y_train, batch_size=16),
                    epochs=100,
                    callbacks=callbacks,
                    validation_data=(X_test, y_test))

model_19.evaluate(X_test, y_test)


Epoch 1/100
10/10 [==============================] - 1s 26ms/step - loss: 24.1407 - accuracy: 0.1586 - val_loss: 7.0775 - val_accuracy: 0.2973 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 0s 17ms/step - loss: 7.1407 - accuracy: 0.2207 - val_loss: 6.1672 - val_accuracy: 0.2973 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 0s 17ms/step - loss: 6.2072 - accuracy: 0.1931 - val_loss: 5.9531 - val_accuracy: 0.2703 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 0s 17ms/step - loss: 5.9052 - accuracy: 0.1379 - val_loss: 5.8152 - val_accuracy: 0.1081 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 0s 17ms/step - loss: 5.7635 - accuracy: 0.2207 - val_loss: 5.6920 - val_accuracy: 0.1351 - lr: 0.0010
Epoch 6/100
10/10 [==============================] - 0s 17ms/step - loss: 5.6480 - accuracy: 0.2138 - val_loss: 5.6001 - val_accuracy: 0.1622 - lr: 0.0010
Epoch 7/100
10/10 [==============================] - 0s 16ms/step - l

[3.8613603115081787, 0.21621622145175934]

## Model 5

In [ ]:
from keras.layers import BatchNormalization, LeakyReLU

model_20 = Sequential()
model_20.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model_20.add(MaxPooling2D((2, 2)))
model_20.add(BatchNormalization())  # Add BatchNormalization after each Conv layer
model_20.add(Conv2D(64, (3, 3), activation='relu'))
model_20.add(MaxPooling2D((2, 2)))
model_20.add(BatchNormalization())
model_20.add(Conv2D(128, (3, 3), activation='relu'))
model_20.add(MaxPooling2D((2, 2)))
model_20.add(BatchNormalization())
model_20.add(Flatten())
model_20.add(Dense(256))
model_20.add(LeakyReLU(alpha=0.1))  # LeakyReLU activation
model_20.add(Dropout(0.5))
model_20.add(Dense(128))
model_20.add(LeakyReLU(alpha=0.1))
model_20.add(Dense(y_train.shape[1], activation='softmax'))

# Compile the model
model_20.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model with adjusted architecture
es = EarlyStopping(patience=10, restore_best_weights=True)
history = model_20.fit(datagen.flow(X_train, y_train, batch_size=64),
                    epochs=150,  # Train for more epochs
                    callbacks=es,
                    validation_data=(X_test, y_test))


model_20.evaluate(X_test, y_test)


Epoch 1/150
3/3 [==============================] - 1s 95ms/step - loss: 2.5536 - accuracy: 0.2207 - val_loss: 25.1510 - val_accuracy: 0.1351
Epoch 2/150
3/3 [==============================] - 0s 76ms/step - loss: 2.5058 - accuracy: 0.2690 - val_loss: 5.3763 - val_accuracy: 0.1081
Epoch 3/150
3/3 [==============================] - 0s 44ms/step - loss: 2.7093 - accuracy: 0.2690 - val_loss: 7.7619 - val_accuracy: 0.2162
Epoch 4/150
3/3 [==============================] - 0s 48ms/step - loss: 2.5523 - accuracy: 0.3241 - val_loss: 4.0743 - val_accuracy: 0.2432
Epoch 5/150
3/3 [==============================] - 0s 45ms/step - loss: 2.7425 - accuracy: 0.2621 - val_loss: 5.8738 - val_accuracy: 0.1622
Epoch 6/150
3/3 [==============================] - 0s 74ms/step - loss: 2.5915 - accuracy: 0.2138 - val_loss: 4.7102 - val_accuracy: 0.1622
Epoch 7/150
3/3 [==============================] - 0s 55ms/step - loss: 2.0056 - accuracy: 0.3586 - val_loss: 3.5605 - val_accuracy: 0.1622
Epoch 8/150
3/3 [==

[1.5310646295547485, 0.37837839126586914]